In [2]:
def DynamicrProgramming(x, y):
    """Retuns the edit distance between 2 strings"""
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    #for i in range(len(y)+1):
    #    D[0][i] = i
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return (min(D[len(x)-1]))

In [3]:
#Testing the Dynamic Programing algorithm
P ="GCGTATGC"
T ="TATTGGCTATACGGT"
DynamicrProgramming(P,T)

2

In [9]:
#Modified overlap module to the get the pair of all reads that have a minimum overlap of 2 and kmer based checking
def readFastq(filename):
    """Reads the Fasta file and returns the reads and read quality"""
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

def overlap_all_pairs(reads, k): 
    """Returns all the pairs of reads that have an overlap of k between them"""
    kmer_dict = {}
    for read in reads:
        for i in range(len(read) - k + 1):
            kmer = read[i:i+k]  # extract the k-mer
            if kmer not in kmer_dict:
                kmer_dict[kmer] = set()  # create a new set if the k-mer is not in the dict
            kmer_dict[kmer].add(read)  # add the read to the set for that k-mer
    
    olaps = [] # set of overlaps
    for read in reads:
        a= read[-k:]
        if a in kmer_dict:
            b_set= kmer_dict[a]
            for b in b_set:
                if b != read:
                    olen = overlap(read, b, k)
                    if olen > 0:
                        olaps.append((read, b))
    return olaps, kmer_dict

In [8]:
#Testing the overlap_all_pairs function to get the list of all overlaps in a set of strings with minimum overlap of k
reads = ['ABCDE', 'CDEFG', 'EFGHI', 'GHIJK','EFGLK']
overlap_pairs,kmer_dict = overlap_all_pairs(reads, 3)
print("The strings that have a minimum overlap of 3 are",overlap_pairs)
print ("No of edges in the overlap graph :", len(overlap_pairs))

#To get the number of nodes with outgoing edges:
reads = ['ABCDE', 'CDEFG', 'EFGHI', 'GHIJK','EFGLK']
overlap_pairs,kmer_dict = overlap_all_pairs(reads, 3)
print("The strings that have a minimum overlap of 3 are",overlap_pairs)
print ("No of edges in the graph :", len(overlap_pairs))
reads_with_outgoing_edges = (set(pair[0] for pair in overlap_pairs))
len(reads_with_outgoing_edges)

The strings that have a minimum overlap of 3 are [('ABCDE', 'CDEFG'), ('CDEFG', 'EFGHI'), ('CDEFG', 'EFGLK'), ('EFGHI', 'GHIJK')]
No of edges in the overlap graph : 4
The strings that have a minimum overlap of 3 are [('ABCDE', 'CDEFG'), ('CDEFG', 'EFGHI'), ('CDEFG', 'EFGLK'), ('EFGHI', 'GHIJK')]
No of edges in the graph : 4


3

In [10]:
!curl -o ERR266411_1.for_asm.fastq https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq

seqs, qual = readFastq('ERR266411_1.for_asm.fastq')

overlap_pairs,kmer_dict = overlap_all_pairs(seqs, 30)
print ("No of edges in the graph :", len(overlap_pairs))

reads_with_outgoing_edges = set(pair[0] for pair in overlap_pairs)
print ("No of nodes with outgoing edges in the graph :", len(reads_with_outgoing_edges))

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2502k  100 2502k    0     0  8121k      0 --:--:-- --:--:-- --:--:-- 8287k


No of edges in the graph : 904746
No of nodes with outgoing edges in the graph : 7161
